In [2]:
!pip install langchain langchain_community langchain_openai chromadb pypdf langsmith qdrant-client

In [12]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Qdrant
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import YoutubeLoader
from langchain_openai import OpenAIEmbeddings
import pandas as pd
import nest_asyncio
import os
import getpass
import pprint

In [4]:
pdf_paths = ["/Users/xico/AIMakerSpace-Midterm/AI_Risk_Management_Framework.pdf",
"/Users/xico/AIMakerSpace-Midterm/Blueprint-for-an-AI-Bill-of-Rights.pdf"]

In [5]:
pdf_documents = []
for pdf_path in pdf_paths:
    loader = PyPDFLoader(pdf_path)
    pdf_documents.extend(loader.load())

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
    )
pdf_docs = text_splitter.split_documents(pdf_documents)

In [8]:
len(pdf_docs)

524

In [9]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

In [13]:
embedding = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = Qdrant.from_documents(
    documents=pdf_docs,
    embedding=embedding,
    location=":memory:",
    collection_name="Full Content"
)

In [14]:
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 4, "fetch_k": 10},
)
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    streaming=True,
)

In [29]:
custom_template = """
You are an expert in artificial intelligence policy, ethics, and industry trends. Your task is to provide clear and accurate answers to questions related to AI's role in politics, government regulations, and its ethical implications for enterprises. Use reliable and up-to-date information from government documents, industry reports, and academic research to inform your responses. Make sure to consider how AI is evolving, especially in relation to the current political landscape, and provide answers in a way that is easy to understand for both AI professionals and non-experts.

For each question:

Provide a summary of the most relevant insights from industry trends and government regulations.
Mention any government agencies, regulations, or political initiatives that play a role in AI governance.
Explain potential ethical concerns and how enterprises can navigate them.
Use real-world examples when possible to illustrate your points.
Here are a few example questions you might receive:

How are governments regulating AI, and what new policies have been implemented?
What are the ethical risks of using AI in political decision-making?
How can enterprises ensure their AI applications meet government ethical standards?

One final rule for you to remember. You CANNOT under any circumstance, answer any question that does not pertain to the AI. If you do answer an out-of-scope question, you could lose your job. If you are asked a question that does not have to do with AI, you must say: "I'm sorry, I don't know the answer to that question."
Context: {context}
Chat History: {chat_history}
Human: {question}
AI:"""

In [30]:
PROMPT = PromptTemplate(
    template=custom_template, input_variables=["context", "question", "chat_history"]
)

In [93]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")

In [32]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.runnable import RunnableLambda

In [33]:
from operator import itemgetter

In [94]:
qa_chain = ConversationalRetrievalChain.from_llm(
        llm,
        retriever=retriever,
        memory=memory,
        combine_docs_chain_kwargs={"prompt": PROMPT},
        return_source_documents=True,
    )

In [67]:
import pprint

In [84]:
response = qa_chain.invoke({
    "question": "How is AI being used in government services?",
    "chat_history": []
})
chat_history = response['chat_history']
source_documents = response['source_documents']
response = response['answer']

In [95]:
response = qa_chain.invoke({
    "question": "What are Trustworthy AI Characteristics?",
    "chat_history": []
})
chat_history = response['chat_history']
source = response['source_documents'][0].metadata['source']
page_number = str(response['source_documents'][0].metadata['page'])
response = response['answer'] + f"\nSource: {source} Page number: {page_number}"

In [96]:
pprint.pprint(response)

('Trustworthy AI characteristics refer to the essential attributes that ensure '
 'artificial intelligence systems operate in a manner that is ethical, '
 'reliable, and aligned with societal values. These characteristics include:\n'
 '\n'
 '1. **Accountable and Transparent**: AI systems should be designed to allow '
 'for accountability in their decision-making processes. This means that the '
 'rationale behind AI decisions should be clear and understandable to users '
 'and stakeholders.\n'
 '\n'
 '2. **Safe**: AI systems must be developed and deployed in a way that '
 'minimizes risks to users and society. This includes ensuring that systems '
 'are robust against failures and adversarial attacks.\n'
 '\n'
 '3. **Valid and Reliable**: AI systems should produce consistent and accurate '
 'results across various scenarios. This involves rigorous testing and '
 'validation to ensure that the AI performs as intended.\n'
 '\n'
 '4. **Interpretable and Explainable**: Users should be able

In [85]:
source_documents[0].metadata.keys()

dict_keys(['source', 'page', '_id', '_collection_name'])